In [1]:
from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmcv import Config
from mmaction.models import build_recognizer
from mmcv.runner import load_checkpoint

In [7]:
mmaction_path = '/ext_data2/comvis/khanhdtq/mmaction2'
work_dir = '/ext_data2/comvis/khanhdtq/slowonly_k400_pretrained_r50_8x4x1_40e_hmdb51_rgb'
data_root = f'{mmaction_path}/data/hmdb51/rawframes'
ann_file_train = f'{mmaction_path}/data/hmdb51/hmdb51_train_split_1_rawframes.txt'
ann_file_val = f'{mmaction_path}/data/hmdb51/hmdb51_val_split_1_rawframes.txt'

cfg = Config.fromfile('configs/hmdb51_rgb_k400_pretrained.py')
cfg.omnisource = False # dirty fix 
# update config's paths according to args
cfg.data.train.data_prefix = data_root 
cfg.data.val.data_prefix = data_root
cfg.work_dir = work_dir 
cfg.data.train.ann_file = ann_file_train
cfg.data.val.ann_file = ann_file_val 
cfg.gpu_ids = range(1) #TODO

In [41]:
# model 
ckpt_path = work_dir + '/epoch_35.pth'
cfg.model.backbone.pretrained = None
model = build_recognizer(cfg.model, test_cfg=cfg.get('test_cfg'))
load_checkpoint(model, ckpt_path, map_location='cpu')
model.cfg = cfg
model.to('cuda')
model.eval()

Use load_from_local loader


Recognizer3D(
  (backbone): ResNet3dSlowOnly(
    (conv1): ConvModule(
      (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
      (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    (pool2): MaxPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck3d(
        (conv1): ConvModule(
          (conv): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        )
        (conv2): ConvModule(
          (conv): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (bn): BatchNorm3d(6

In [46]:
state_dict = torch.load(work_dir + '/epoch_35.pth')['state_dict']

In [47]:
state_dict

OrderedDict([('backbone.conv1.conv.weight',
              tensor([[[[[ 2.1557e-02,  4.0029e-02,  4.7194e-02,  ...,  6.2699e-02,
                           5.1689e-02,  5.4744e-02],
                         [-4.2717e-03, -5.9640e-03, -1.1328e-02,  ..., -1.1731e-02,
                          -2.4180e-02, -2.4312e-02],
                         [ 8.1723e-03,  1.1170e-02,  2.8133e-03,  ...,  4.8348e-03,
                          -6.7734e-03, -1.1520e-02],
                         ...,
                         [ 4.1460e-02,  4.6606e-02,  3.9197e-02,  ...,  5.0393e-02,
                           4.5866e-02,  4.3387e-02],
                         [ 4.5930e-02,  5.2975e-02,  4.5716e-02,  ...,  6.0485e-02,
                           5.8632e-02,  5.8885e-02],
                         [ 4.8526e-02,  5.8777e-02,  5.1712e-02,  ...,  6.2966e-02,
                           6.0618e-02,  6.1198e-02]]],
              
              
                       [[[ 5.4854e-02,  4.6658e-02,  3.8855e-02,  ...,  

In [48]:
from mmcv.runner import load_state_dict

cfg.model.backbone.pretrained = None
model = build_recognizer(cfg.model, test_cfg=cfg.get('test_cfg'))
load_state_dict(model, state_dict)
model.cfg = cfg
model.to('cuda')
model.eval()

Recognizer3D(
  (backbone): ResNet3dSlowOnly(
    (conv1): ConvModule(
      (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
      (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    (pool2): MaxPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck3d(
        (conv1): ConvModule(
          (conv): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        )
        (conv2): ConvModule(
          (conv): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (bn): BatchNorm3d(6

In [9]:
# test dataset
val_dataset = build_dataset(cfg.data.val)

In [11]:
val_dataset[0].keys()

dict_keys(['imgs', 'label'])

In [14]:
val_dataset[0]['imgs'].shape #(bz, 3, T, H, W) --> T x H x W x 3

torch.Size([1, 3, 8, 224, 224])

In [15]:
val_dataset[0]['label']

47

In [30]:
# from mmaction.apis import inference_recognizer
# res = inference_recognizer(model, val_dataset[0])
# data = {'imgs': val_dataset[0]['imgs'][None].cuda(), 'label': val_dataset[0]['label']}
data = {'imgs': val_dataset[0]['imgs'][None].cuda()}
import torch 
with torch.no_grad():
    scores = model(return_loss=False, **data)[0]

In [31]:
len(scores)

51

In [32]:
import numpy as np 
np.argmax(scores)

33

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [49]:
from tqdm.notebook import tqdm 
labels = []
scores = []
losses = []
num_samples = len(val_dataset)
with torch.no_grad():
    for i in tqdm(range(num_samples), total=num_samples):
        data = {'imgs': val_dataset[i]['imgs'][None].to(device)}
        score = model(return_loss=True, **data)[0]
        scores.append(score)
        labels.append(val_dataset[i]['label'])

  0%|          | 0/1530 [00:00<?, ?it/s]

In [50]:
from mmaction.core.evaluation import top_k_accuracy 
top_k_accuracy(scores, labels, topk=(1, 5)) # epoch 40: 0.6882352941176471, 0.9169934640522875

[0.6843137254901961, 0.9130718954248366]

In [58]:
model.forward_test??